In [62]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = sns.load_dataset("anscombe")
df.head()

,dataset,x,y
0,I,10.0,8.04
1,I,8.0,6.95
2,I,13.0,7.58
3,I,9.0,8.81
4,I,11.0,8.33


In [65]:
# df["dataset"].unique()

# df.query("dataset == 'III'")

# import plotly.express as px

# px.scatter(
#     df.query("dataset == 'II'"),
#     x="x",
#     y="y"
#     )

In [68]:
import socket
import psutil
import os
from dash import Dash, html, dcc
from dash.dependencies import Input, Output

DEFAULT_PORT = 8050  # Porta padrão

def find_free_port():
    """Tenta usar a porta padrão; se estiver ocupada, encontra uma livre."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        if s.connect_ex(("127.0.0.1", DEFAULT_PORT)) != 0:
            return DEFAULT_PORT  # Porta padrão está livre, então usa ela.

    # Se a porta padrão estiver ocupada, busca uma nova
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("127.0.0.1", 0))  # Pega uma porta livre automaticamente
        return s.getsockname()[1]  # Retorna o número da porta disponível

def kill_previous_process(port):
    """Encerra processos rodando na porta especificada (Windows compatível)."""
    for proc in psutil.process_iter(attrs=['pid', 'name']):
        try:
            for conn in proc.connections(kind='inet'):
                if conn.laddr.port == port:
                    print(f"Encerrando processo {proc.pid} ({proc.name()}) na porta {port}...")
                    os.kill(proc.pid, 9)  # Mata o processo
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass  # Ignora processos inacessíveis

app = Dash(__name__)

app.layout = html.Div([
    html.H3("Anscombe's Quartet"),
    dcc.Dropdown(id="dropdown", options=df["dataset"].unique()),
    dcc.Graph(
        id="visual",
        figure = px.scatter(
            df.query (f"dataset == 'II' "),
            x="x",
            y="y"
        ))
])

@app.callback(Output("visual", "figure"), Input("dropdown", "value"))
def anscombes_scatter(dataset):
    figure=px.scatter(
        df.query (f"dataset == '{dataset}'"),
        x="x",
        y="y"
        )
    return figure

if __name__ == "__main__":
    port = find_free_port()
    kill_previous_process(port)
    print(f"Rodando na porta {port}")
    app.run(port=port)

C:\Users\bgran\AppData\Local\Temp\ipykernel_28956\3315473517.py:24: DeprecationWarning:

connections() is deprecated and will be removed; use net_connections() instead



Rodando na porta 58478
